In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data
playstoreData = pd.read_excel('GooglePlaystore.xlsx')

# moves outlier in reviews
print("Rows before:", playstoreData.shape[0])
playstoreData = playstoreData[playstoreData['Reviews'] != '3.0M']
print("Rows after:", playstoreData.shape[0])

# remove rows with varies with device
print("Rows before:", playstoreData.shape[0])
playstoreData = playstoreData[~playstoreData.apply(lambda row: row.astype(str).str.contains("Varies with device").any(), axis=1)]
print("Rows after:", playstoreData.shape[0])
# Clean Android Version
def cleanAndroidVersion(version):
    if pd.isnull(version): return None
    version = str(version).split('-')[0].strip().replace('and up', '').strip()
    versionParts = version.split('.')
    return '.'.join(versionParts[:2]) if len(versionParts) > 1 else versionParts[0]

print("Before:", playstoreData['Android Ver'].unique()[:5])
playstoreData['Android Ver'] = playstoreData['Android Ver'].apply(cleanAndroidVersion)
print("After:", playstoreData['Android Ver'].unique()[:5])

# cleans the instals
playstoreData['Installs'] = playstoreData['Installs'].str.replace(',', '').str.replace('+', '')
playstoreData = playstoreData[playstoreData['Installs'].str.isdigit()]
playstoreData['Installs'] = playstoreData['Installs'].astype(int)
print(playstoreData['Installs'].head())
# changes review to int 
playstoreData['Reviews'] = playstoreData['Reviews'].astype(int)

# missing rating
beforeDrop = playstoreData.shape[0]
maskToDrop = (playstoreData['Rating'].isnull()) & (playstoreData['Reviews'] < 100) & (playstoreData['Installs'] < 50000)
playstoreData = playstoreData[~maskToDrop]

print("Dropped rows:", beforeDrop - playstoreData.shape[0])
categoryRatingMean = playstoreData.groupby('Category')['Rating'].transform(lambda x: round(x.mean(), 2))
playstoreData['Rating'] = playstoreData['Rating'].fillna(categoryRatingMean)
print("Remaining missing Ratings:", playstoreData['Rating'].isnull().sum())

#cleans columns
def convertSize(size):
    if pd.isnull(size) or size == '0': return 0
    size = size.strip().upper()
    if size.endswith('M'):
        return int(float(size[:-1]) * 1_000_000)
    elif size.endswith('K'):
        return int(float(size[:-1]) * 1_000)
    try:
        return int(size)
    except:
        return None

playstoreData['Size'] = playstoreData['Size'].apply(convertSize)

# ANALYSIS

# describes category
print("\nCategory-wise Rating Description:")
print(playstoreData.groupby('Category')['Rating'].describe())

# top 3 apps in cat
freeApps = playstoreData[playstoreData['Type'] == 'Free']

def topThreeAppsBy(columnName):
    return (freeApps.sort_values(columnName)
                   .groupby('Category')
                   .head(3)[['Category', 'App', columnName]])

print("\nTop 3 Free Apps by Rating:")
print(topThreeAppsBy('Rating'))

print("\nTop 3 Free Apps by Installs:")
print(topThreeAppsBy('Installs'))

print("\nTop 3 Free Apps by Reviews:")
print(topThreeAppsBy('Reviews'))

# stats of paid apps
paidApps = playstoreData[playstoreData['Type'] == 'Paid'].copy()
paidApps['Price'] = paidApps['Price'].astype(str).str.replace(r'[$]', '', regex=True).astype(float)
print("\nPaid App Prices - Avg, Max, Min:")
print(paidApps['Price'].agg(['mean', 'max', 'min']))

# VISUALIZATIONS
# pie chart 
genresExploded = playstoreData.assign(Genres=playstoreData['Genres'].str.split(';')).explode('Genres')
genreCounts = genresExploded['Genres'].value_counts()
genreCounts.plot(kind='pie', autopct='%1.1f%%', figsize=(10, 10), title='Genre Distribution')
plt.ylabel('')
plt.tight_layout()
plt.show()

#Box plot
subset = playstoreData[playstoreData['Category'].isin(['BUSINESS', 'EDUCATION'])]
subset.boxplot(column='Rating', by='Category', grid=False)
plt.title('Ratings by Category')
plt.suptitle('')
plt.xlabel('Category')
plt.ylabel('Rating')
plt.tight_layout()
plt.show()